In [1]:
import sys

sys.path.append("/home/jupyter-aamir09/mlops2_with_dagster/")


from mlops2_with_dagster import encoder_pipeline
from pathlib import Path
from joblib import dump, load
from hamilton import driver, base
from mlops2_with_dagster.utils import get_project_dir, printse

from sklearn.preprocessing import (
    StandardScaler,
    LabelEncoder
)
from ortho.utils.logger import Logger
from ortho.ortho.decorators.task import task
from typing import Tuple

import pandas as pd 
import mlflow as mf
from ortho.ortho.callbacks.mlflow import MlFlowCallBack
from ortho.ortho.callbacks.logger import LoggerCallBack
import os


logger = Logger().logger()

   

        
        

None


2023-08-30 12:37:51.234607: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-30 12:37:51.266881: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-30 12:37:51.267673: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-30 12:37:51,741 - DEBUG - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
2023-08-30 12:37:51.793885: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-30 12:37:51,919 - DEBUG - Creating converter from 7 to 5
2023-08-30 12:37:51,920 - DEBUG - Creating converter from 

In [2]:
project = 'mlops2_with_dagster'
project_dir = get_project_dir(project)
printse(f'project_dir: {project_dir}')

>>>> project_dir: /home/jupyter-aamir09/mlops2_with_dagster


In [3]:
class TargetExtractor:
    
    def __init__(self, callbacks,  experiment_name, run_name, load_from_artifact=False):
        self.load_from_artifact = load_from_artifact
        self.callbacks = callbacks
        self.experiment_name = experiment_name
        self.run_name = run_name
        os.environ["MLFLOW_EXPERIMENT_NAME"] = self.experiment_name
        os.environ["MLFLOW_RUN_NAME"] = self.run_name
        
        self.index_col = 'passengerid'
        self.target_col = "survived"
        self.cat_cols = ["sex", "cabin", "embarked"]
        self.config = {
            'index_column': self.index_col,
            'target_column': self.target_col,
            'categorical_columns': self.cat_cols
        }
    
    @task(build_on_previous_run=True, end_mlflow_run=False)       
    def get_target(self, df_train):
        encode_dr = driver.Driver(self.config, encoder_pipeline)
        output_nodes = ['target']
        
        out = encode_dr.execute(output_nodes ,
        inputs = dict(
            df_train = df_train
        )         
        )
        
        data = {"target": out }
        payload = {"artifact_path": "/home/jupyter-aamir09/mlops2_with_dagster/artifacts/mlflow_artfacts"}
        
        return data, payload
    
    def run(self, *args, **kwargs):
        
        return self.get_target(*args, **kwargs)
        
        
    

In [4]:
train_data : Path = project_dir/"data/train.csv"
test_data : Path = project_dir/"data/test.csv"

In [5]:
# parameters
df_train = pd.read_csv(train_data)

In [ ]:
import dagstermill
extractor = TargetExtractor(experiment_name="Mlflow_with_Dagster",
                          run_name="BadamBhum",
                          callbacks = [LoggerCallBack("/home/jupyter-aamir09/mlops2_with_dagster/notebooks", kernel_names=["get_target"]),
                                       MlFlowCallBack(kernel_names=["get_target"])])

# mf.end_run()
dagstermill.yield_result(extractor.run(df_train=df_train)[0]["target"], output_name="target")

2023-08-30 12:37:52,456 - DEBUG - Starting new HTTP connection (1): 127.0.0.1:5002
2023-08-30 12:37:52,462 - DEBUG - http://127.0.0.1:5002 "GET /api/2.0/mlflow/experiments/get-by-name?experiment_name=Mlflow_with_Dagster HTTP/1.1" 200 241
2023-08-30 12:37:52,464 - DEBUG - Resetting dropped connection: 127.0.0.1
2023-08-30 12:37:52,469 - DEBUG - http://127.0.0.1:5002 "GET /api/2.0/mlflow/runs/get?run_uuid=1f0a6e058efb4c5d9e9fbecc36928d78&run_id=1f0a6e058efb4c5d9e9fbecc36928d78 HTTP/1.1" 200 1261
2023-08-30 12:37:52,471 - DEBUG - Resetting dropped connection: 127.0.0.1
2023-08-30 12:37:52,482 - DEBUG - http://127.0.0.1:5002 "POST /api/2.0/mlflow/runs/update HTTP/1.1" 200 423
2023-08-30 12:37:52,484 - DEBUG - Resetting dropped connection: 127.0.0.1
2023-08-30 12:37:52,488 - DEBUG - http://127.0.0.1:5002 "GET /api/2.0/mlflow/runs/get?run_uuid=1f0a6e058efb4c5d9e9fbecc36928d78&run_id=1f0a6e058efb4c5d9e9fbecc36928d78 HTTP/1.1" 200 1260
2023-08-30 12:37:52,489 - INFO - Successfuly initiated run

['self', 'df_train']
{}


,target
passengerid,
0,1
1,0
2,0
3,0
4,1
...,...
99995,1
99996,0
99997,0


2023-08-30 12:37:52,738 - DEBUG - Succeed in sending telemetry consisting of [b'{"api_key": "phc_mZg8bkn3yvMxqvZKRlMlxjekFU5DFDdcdAsijJ2EH5e", "event": "os_hamilton_run_start", "properties": {"os_type": "posix", "os_version": "Linux-5.4.0-124-generic-x86_64-with-glibc2.31", "python_version": "3.10.12/CPython", "distinct_id": "e4e272c1-5845-4b12-8bd8-09d84d585dbc", "hamilton_version": [1, 27, 2], "telemetry_version": "0.0.1", "number_of_nodes": 32, "number_of_modules": 1, "number_of_config_items": 3, "decorators_used": {"parameterize_sources": 4, "extract_columns": 2, "extract_fields": 1}, "graph_adapter_used": "hamilton.base.SimplePythonDataFrameGraphAdapter", "result_builder_used": "hamilton.base.PandasDataFrameResult", "driver_run_id": "adc19a70-ca89-4225-9662-764512c0df14", "error": null, "graph_executor_class": "DefaultGraphExecutor"}}'].
2023-08-30 12:37:52,761 - DEBUG - Succeed in sending telemetry consisting of [b'{"api_key": "phc_mZg8bkn3yvMxqvZKRlMlxjekFU5DFDdcdAsijJ2EH5e", "e